In [2]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [3]:
df = pd.read_csv('google_books.csv')
print(df.shape)
df.head()

(1116, 14)


,title,subtitle,authors,publisher,month,year,categories,pageCount,listPrice,images,isEbook,ratingsCount,averageRating,description
0,The Litigators,NaN,John Grisham,Dell,10,2011,Fiction,496,9.99,False,True,137,3.5,The partners at Finley & Figg often refer to t...
1,Ragtime,A Novel,E.L. Doctorow,Random House,11,2010,Fiction,336,14.99,True,True,57,3.5,Selected by the Modern Library as one of the 1...
2,Her Perfect Man,NaN,Mary Lynn Baxter,Silhouette,5,2011,Fiction,192,3.99,False,True,1,5.0,One look at firebrand Katherine Mays and Bryce...
3,The Vanished Man,A Lincoln Rhyme Novel,Jeffery Deaver,Simon and Schuster,8,2012,Fiction,577,NaN,False,False,19,4.0,Hunting down a killer and master illusionist w...
4,One Sunday,A Novel,Carrie Gerlach Cecil,Simon and Schuster,2,2013,Fiction,255,NaN,False,False,6,4.0,"When her unborn child's health is put at risk,..."


In [4]:
df.dropna(subset=['description'], inplace=True)
df.reset_index(inplace=True, drop=True)
df.shape
df

,title,subtitle,authors,publisher,month,year,categories,pageCount,listPrice,images,isEbook,ratingsCount,averageRating,description
0,The Litigators,NaN,John Grisham,Dell,10,2011,Fiction,496,9.99,False,True,137,3.5,The partners at Finley & Figg often refer to t...
1,Ragtime,A Novel,E.L. Doctorow,Random House,11,2010,Fiction,336,14.99,True,True,57,3.5,Selected by the Modern Library as one of the 1...
2,Her Perfect Man,NaN,Mary Lynn Baxter,Silhouette,5,2011,Fiction,192,3.99,False,True,1,5.0,One look at firebrand Katherine Mays and Bryce...
3,The Vanished Man,A Lincoln Rhyme Novel,Jeffery Deaver,Simon and Schuster,8,2012,Fiction,577,NaN,False,False,19,4.0,Hunting down a killer and master illusionist w...
4,One Sunday,A Novel,Carrie Gerlach Cecil,Simon and Schuster,2,2013,Fiction,255,NaN,False,False,6,4.0,"When her unborn child's health is put at risk,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,Abstract Algebra: An Introduction,NaN,Thomas W. Hungerford,Cengage Learning,7,2012,Mathematics,616,69.49,True,True,2,3.0,Abstract Algebra: An Introduction is set apart...
1107,Introduction to Geophysical Fluid Dynamics,Physical and Numerical Aspects,"Benoit Cushman-Roisin, Jean-Marie Beckers",Academic Press,1,2011,Mathematics,828,NaN,True,False,4,5.0,Inhaltsverzeichnis: I Fundamentals - 1: Introd...
1108,Cambridge Primary Mathematics Stage 5 Learner'...,NaN,"Emma Low, Mary Wood",Cambridge University Press,5,2014,Mathematics,129,NaN,True,False,1,5.0,Written for use with the Cambridge Primary Mat...
1109,Mathematical Statistics with Applications,NaN,"Dennis Wackerly, William Mendenhall, Richard L...",Cengage Learning,10,2014,Mathematics,944,NaN,False,False,2,3.5,In their bestselling MATHEMATICAL STATISTICS W...


In [5]:
count_vectorizer = CountVectorizer()

In [6]:
bag_of_words = count_vectorizer.fit_transform(df['description'].dropna())

In [7]:
feature_names = count_vectorizer.get_feature_names()

In [8]:
bag = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)

In [9]:
bag

,000,000000,0001,007,0245,05,08,0px,10,100,...,वम,वस,वह,शन,शब,षय,सकत,सत,सर,सहज
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1107,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1108,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1109,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
bag.columns

Index(['000', '000000', '0001', '007', '0245', '05', '08', '0px', '10', '100',
       ...
       'वम', 'वस', 'वह', 'शन', 'शब', 'षय', 'सकत', 'सत', 'सर', 'सहज'],
      dtype='object', length=21536)

In [11]:
from nltk.stem import WordNetLemmatizer
lemmer = WordNetLemmatizer()
new_corpus = [' '.join([lemmer.lemmatize(word) for word in text.split(' ')]) for text in df['description'].dropna()]
new_corpus[:5]

['The partner at Finley & Figg often refer to themselves a a “boutique law firm.” Boutique, a in chic, selective, and prosperous. Oscar Finley and Wally Figg are none of these things. They are a two-bit operation of ambulance chaser who bicker like an old married couple. Until change come their way—or, more accurately, stumble in. After leaving a fast-track career and going on a serious bender, David Zinc is sober, unemployed, and desperate enough to take a job at Finley & Figg. Now the firm is ready to tackle a case that could make the partner rich—without requiring them to actually practice much law. A class action suit ha been brought against Varrick Labs, a pharmaceutical giant with annual sale of $25 billion, alleging that Krayoxx, it most popular drug, cause heart attacks. Wally smell money. All Finley & Figg ha to do is find a handful of Krayoxx user to join the suit. It almost seems too good to be true . . . and it is. Includes an excerpt of John Grisham’s Calico Joe and a spec

## Count Vectorizer

In [12]:
count_vect = CountVectorizer(stop_words='english')
values = count_vect.fit_transform(df['description'].dropna())
feature_names = count_vect.get_feature_names()
count = pd.DataFrame(values.toarray(), columns = feature_names)

In [13]:
count_scores = count.apply(np.sum, axis=0)

In [14]:
count_scores

000       44
000000     4
0001       1
007        1
0245       1
          ..
षय         1
सकत        1
सत         1
सर         1
सहज        1
Length: 21247, dtype: int64

In [15]:
count_idx = count_scores.sort_values(ascending=False)[0:400].index

In [16]:
filtered = count[count_idx]
count_one_hot_words = filtered.applymap(lambda x: 1 if x > 0 else 0)

In [17]:
count_one_hot_words.apply(np.sum, axis = 0)

new            459
book           433
world          393
life           352
data            78
              ... 
class           35
london          31
illustrated     36
acclaimed       39
philosophy      25
Length: 400, dtype: int64

In [18]:
count_one_hot_words

,new,book,world,life,data,war,times,story,author,history,...,filled,british,france,land,wide,class,london,illustrated,acclaimed,philosophy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,0,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1107,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1108,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1109,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
## this does the same thing but with bigrams, it failed to improve model performance

count_vect = CountVectorizer(stop_words='english', ngram_range=(2, 2))
values = count_vect.fit_transform(df['description'].dropna())
feature_names = count_vect.get_feature_names()
count = pd.DataFrame(values.toarray(), columns = feature_names)

count_scores = count.apply(np.sum, axis=0)
count_idx = count_scores.sort_values(ascending=False)[0:400].index

filtered = count[count_idx]
count_one_hot_words_bigram = filtered.applymap(lambda x: 1 if x > 0 else 0)
count_one_hot_words_bigram.apply(np.sum, axis = 0)
count_one_hot_words_bigram.to_csv('count_one_hot_words_bigram.csv')

## TF-IDF Vectorizer

In [20]:
## i am taking the mean tf-idf value for each feature/word in the dataframe
## after that, sorting them them in descending order so highest mean values are first
## then, i am taking the 400 features/words with the highest tf-idf mean-value
## then i am one-hot-encoding these 400 features and putting them back in the dataframe

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
values = tfidf_vectorizer.fit_transform(df['description'].dropna())
feature_names = tfidf_vectorizer.get_feature_names()
tfidf = pd.DataFrame(values.toarray(), columns = feature_names)

In [21]:
words_scores = tfidf.apply(np.mean, axis=0)

In [22]:
words_scores

000       0.003366
000000    0.000211
0001      0.000108
007       0.000079
0245      0.000108
            ...   
षय        0.000084
सकत       0.000084
सत        0.000084
सर        0.000084
सहज       0.000084
Length: 21247, dtype: float64

In [23]:
words_idx = words_scores.sort_values(ascending=False)[0:400].index

In [24]:
filtered = tfidf[words_idx]
tfidf_one_hot_words = filtered.applymap(lambda x: 1 if x > 0 else 0)

In [25]:
tfidf_one_hot_words.apply(np.sum, axis = 0)

new           459
book          433
life          352
world         393
data           78
             ... 
greatest       36
words          36
gives          43
university     27
wide           40
Length: 400, dtype: int64

In [26]:
tfidf_one_hot_words

,new,book,life,world,data,war,history,story,author,edition,...,cookbook,born,voice,music,behavior,greatest,words,gives,university,wide
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1107,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1108,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1109,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
pd.concat([df, tfidf_one_hot_words], axis = 1).head()

,title,subtitle,authors,publisher,month,year,categories,pageCount,listPrice,images,...,cookbook,born,voice,music,behavior,greatest,words,gives,university,wide
0,The Litigators,NaN,John Grisham,Dell,10,2011,Fiction,496,9.99,False,...,0,0,0,0,0,0,0,0,0,0
1,Ragtime,A Novel,E.L. Doctorow,Random House,11,2010,Fiction,336,14.99,True,...,0,0,0,0,0,0,0,0,0,0
2,Her Perfect Man,NaN,Mary Lynn Baxter,Silhouette,5,2011,Fiction,192,3.99,False,...,0,0,0,0,0,0,0,0,0,0
3,The Vanished Man,A Lincoln Rhyme Novel,Jeffery Deaver,Simon and Schuster,8,2012,Fiction,577,NaN,False,...,0,0,0,1,0,0,0,0,0,0
4,One Sunday,A Novel,Carrie Gerlach Cecil,Simon and Schuster,2,2013,Fiction,255,NaN,False,...,0,0,0,0,0,0,0,0,0,0


In [234]:
count_one_hot_words.to_csv('count_one_hot_words.csv', index=False)
tfidf_one_hot_words.to_csv('tfidf_one_hot_words.csv', index=False)